In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [ ]:
TRAIN_PATH = "/kaggle/input/playground-series-s5e11/train.csv"
TEST_PATH  = "/kaggle/input/playground-series-s5e11/test.csv"
SAMPLE_SUB_PATH = "/kaggle/input/playground-series-s5e11/sample_submission.csv"

In [ ]:
train=pd.read_csv(TRAIN_PATH)
test=pd.read_csv(TEST_PATH)
sample_sub=pd.read_csv(SAMPLE_SUB_PATH)


In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
target = "loan_paid_back"

# Features & target
x = train.drop(columns=[target])
y = train[target].astype(int)

In [ ]:
cat_cols = x.select_dtypes(include=["object", "category"]).columns.tolist()
print("Categorical columns:", cat_cols)



In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x,y,random_state=42,stratify=y,test_size=0.2)

In [ ]:
train_pool=Pool(x_train,y_train,cat_features=cat_cols)
test_pool=Pool(test,cat_features=cat_cols)
val_pool=Pool(x_val,y_val,cat_features=cat_cols)

In [ ]:
model = CatBoostClassifier(
    iterations=3000,        
    learning_rate=0.03,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    od_type="Iter",          
    od_wait=200,
    verbose=200
)

model.fit(train_pool, eval_set=val_pool)


In [ ]:
full_pool = Pool(x, y, cat_features=cat_cols)

final_model = CatBoostClassifier(
    iterations=model.tree_count_,   
    learning_rate=0.03,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=100
)

final_model.fit(full_pool)

In [ ]:
test_pool = Pool(test, cat_features=cat_cols)
test_pred = final_model.predict_proba(test_pool)[:, 1]

In [ ]:

submission = sample_sub.copy()
submission["loan_paid_back"] = test_pred

submission.to_csv("submission.csv", index=False)
submission.head()
